In [1]:
import numpy as np


In [2]:
from typing import List


class Solution:
    def minCost(self, nums: List[int], cost: List[int]) -> int:
        def get_cost(base):
            return sum(abs(base - num) * c for num, c in zip(nums, cost))

        # Initialize the left and the right boundary of the binary search.
        left, right = min(nums), max(nums)
        answer = get_cost(nums[0])

        # As shown in the previous picture, if F(mid) > F(mid + 1), then the minimum
        # is to the right of mid, otherwise, the minimum is to the left of mid.
        while left < right:
            mid = (left + right) // 2
            cost_1 = get_cost(mid)
            cost_2 = get_cost(mid + 1)
            answer = min(cost_1, cost_2)

            if cost_1 > cost_2:
                left = mid + 1
            else:
                right = mid

        return answer


## Dãy con lập thành cấp số cộng dài nhất


In [3]:
class Solution:
    def longestArithSeqLength(self, nums: List[int]) -> int:
        dp = {}
        for r in range(1, len(nums)):
            for l in range(r):
                diff = nums[r] - nums[l]
                dp[r, diff] = dp.get((l, diff), 1) + 1
        return max(dp.values())


%time Solution().longestArithSeqLength(np.random.randint(0, 500, 1000))


CPU times: total: 438 ms
Wall time: 482 ms


10

In [4]:
from time import time
from collections import defaultdict

np.random.seed(0)
# nums = list(np.random.randint(0, 50, 20))
nums = [24, 33, 23, 23, 29, 24, 31, 29, 36,
        32, 22, 18, 35, 36, 37, 41, 23, 39, 33, 32]
n = len(nums)
s = [0] * (1 << n)
candidates = []
mark = defaultdict(list)
s_num = sum(nums)
start = time()
for i in range(1, 1 << n):
    for j in range(n):
        if i & (1 << j):
            s[i] = s[i ^ (1 << j)] + nums[j]
            if s[i] <= s_num // 2:
                candidates.append(s[i])
            mark[s[i]].append(i)
            break
candidates = sorted(set(candidates), reverse=True)
mark[0] = [0]
ans = 0
for num in candidates:
    if ans != 0:
        break
    for i in mark[s_num - 2*num]:
        for j in mark[num]:
            if i & j == 0:
                ans = num
                break
        if ans != 0:
            break
print(time() - start)
print(ans)


1.577779769897461
300


In [5]:
from functools import lru_cache


class Solution:
    def countRoutes(self, locations: List[int], start: int, finish: int, fuel: int) -> int:
        n = len(locations)
        mod = 10**9 + 7

        @lru_cache(maxsize=None)
        def dfs(i, fuel):
            count = 0
            for j in range(n):
                if i != j:
                    x = fuel - abs(locations[i] - locations[j])
                    if x >= 0:
                        count = (count + dfs(j, x)) % mod
                        if j == finish:
                            count = (count + 1) % mod
            return count
        if start == finish:
            return (dfs(start, fuel) + 1) % mod
        return dfs(start, fuel)


In [6]:
from heapq import heapify, heappop, heappush


class Solution:
    def totalCost(self, costs: List[int], k: int, candidates: int) -> int:
        pq = []
        for i in range(candidates):
            pq.append((costs[i], 0))
        for i in range(max(candidates, len(costs) - candidates), len(costs)):
            pq.append((costs[i], 1))

        heapify(pq)

        answer = 0
        next_head, next_tail = candidates, len(costs) - 1 - candidates

        for _ in range(k):
            cur_cost, cur_section_id = heappop(pq)
            answer += cur_cost
            if next_head <= next_tail:
                if cur_section_id == 0:
                    heappush(pq, (costs[next_head], 0))
                    next_head += 1
                else:
                    heappush(pq, (costs[next_tail], 1))
                    next_tail -= 1

        return answer


In [7]:
class Solution:
    def maxProbability(self, n: int, edges: List[List[int]], succProb: List[float], start: int, end: int) -> float:
        adj = defaultdict(list)
        for i in range(len(edges)):
            adj[edges[i][0]].append((edges[i][1], succProb[i]))
            adj[edges[i][1]].append((edges[i][0], succProb[i]))
        visited = [False] * n
        visited[start] = True
        prob = [0] * n
        prob[start] = 1
        pq = [(0, start)]
        while pq:
            _, node = heappop(pq)
            if visited[node]:
                continue
            visited[node] = True
            for neighbor, p in adj[node]:
                if prob[node] * p > prob[neighbor]:
                    prob[neighbor] = prob[node] * p
                    heappush(pq, (prob[neighbor], neighbor))
        return prob[end]


In [10]:
class Solution:
    def kthFactor(self, n: int, k: int) -> int:
        factors = []
        for i in range(1, int(n**0.5) + 1):
            if n % i == 0:
                factors.append(i)
        x = len(factors)
        if k>2*x:
            return -1
        if k<=x:
            return factors[k-1]
        return n//factors[x-k]
s = Solution()
s.kthFactor(100)

-1